### Import packages

In [1]:
# import dva_sdhdf_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from ipywidgets import interact
import matplotlib.patches as patches


#### Change the directory to where the files are located" ####
day ='25'
directory = '../DVA/Data_Files/DVA_Day_Surveys/'

### Change the scan details to your current scan
# df = 0.5        
# dt = 6.9444431574083865e-06

#TODO: actually do the scan properties definition in the beginning
##############################################################

### Read in the file listing azimuth scan start and stop times:

In [2]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the data and store it in arrays:
with open(directory+'DVAsurvey_phase1_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
        
# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])

# Convert start and stop times to Modified Julian Day (MJD).
# This is needed for plotting and for selecting out data collected
# between particular times:
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd


1052 2022-07-04T03:57:53.110 2022-07-04T04:15:55.184
2565 2022-07-04T04:20:37.406 2022-07-04T04:38:39.310
2566 2022-07-04T04:40:22.266 2022-07-04T04:58:24.003
1271 2022-07-04T05:03:42.716 2022-07-04T05:21:44.818
1272 2022-07-04T05:23:27.291 2022-07-04T05:41:29.292
2857 2022-07-04T05:48:35.269 2022-07-04T06:06:37.317
0124 2022-07-04T06:11:19.190 2022-07-04T06:29:21.101
1637 2022-07-04T06:34:03.920 2022-07-04T06:52:06.005
0342 2022-07-04T06:57:23.897 2022-07-04T07:15:25.930
1927 2022-07-04T07:21:55.804 2022-07-04T07:39:58.034
1928 2022-07-04T07:42:16.530 2022-07-04T08:00:18.319
2001 2022-07-04T08:03:48.814 2022-07-04T08:21:51.062
2002 2022-07-04T08:24:09.373 2022-07-04T08:42:11.545
2075 2022-07-04T08:45:41.821 2022-07-04T09:03:44.188
2148 2022-07-04T09:07:50.625 2022-07-04T09:25:52.832
2149 2022-07-04T09:27:35.229 2022-07-04T09:45:37.044
2150 2022-07-04T09:47:56.026 2022-07-04T10:05:57.828
2151 2022-07-04T10:07:40.752 2022-07-04T10:25:42.662
2152 2022-07-04T10:27:25.410 2022-07-04T10:45:

## Read in scan files and stich them together:

In [3]:
t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []
noise_set = []
trim_flag = []

scan0 = f"{scan_id[0]:04}"

# Use one of the scans to get the list of frequencies:
freq_channel_increment = 1 #TODO: I'll have to change this to 1 once I am sure I'm reading the data correctly

file = h5py.File(directory+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][::freq_channel_increment]/1e6
df = freq[1] - freq[0]

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])

# Loop through all the scans in the "scan_num" list:
for i in scan_id[0:5]:
    print(i)
    # select the file:
    file = h5py.File(directory+'dva_survey_raw_scan_'+f"{i:04}"+'.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state']]) #This is a "mask" for noise regions 1 = noise 0=all good
    trim_flag = np.concatenate([trim_flag,dataset['metadata']['trim_scan_flag']])
    
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,dataset['data'][:,0,::freq_channel_increment]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][:,1,::freq_channel_increment]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][:,2,::freq_channel_increment]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][:,3,::freq_channel_increment]],axis=0)
    
t_plt = Time(t_set, format='isot',scale='utc').mjd

1052
<HDF5 file "dva_survey_raw_scan_1052.h5" (mode r)>
2565
<HDF5 file "dva_survey_raw_scan_2565.h5" (mode r)>
2566
<HDF5 file "dva_survey_raw_scan_2566.h5" (mode r)>
1271
<HDF5 file "dva_survey_raw_scan_1271.h5" (mode r)>
1272
<HDF5 file "dva_survey_raw_scan_1272.h5" (mode r)>


### Polarized Intensity

In [4]:
polarized = []
for i,j in zip(reRL_set,imRL_set):
    PI = np.sqrt((i**2)+(j**2))
    polarized.append(PI)
polarized_set = np.array(polarized)

# NEW RFI_Excision functions (WORK IN PROGRESS)

In [5]:
# power_array has time and frequency
#TODO: Count how long it takes to run this for a single frequency 
#TODO: Change the name of the function below to Find_RFI_Excision_Regions, or make a wrapper function to go through all the frequencies

# def Get_RFI_Excision_Mask(time_array, power_array, time_steps, slope_threshold, starting_frequency): #TODO: The final function will not take starting_frequency as an input, I'll loop through all frequencies
#     confirmed_RFI_events = Find_RFI_Excision_Intervals(time_array, power_array, time_steps, slope_threshold, starting_frequency)
#     mask = []
#     #TODO: Populate mask
#     return mask

def Possible_RFI_Found(time_array, single_freq_power_array, idx, time_slope_threshold):
    last_idx = (len(time_array) - 1)
    possible_RFI_Found = False
    rfi_start_idx = 0
    rfi_end_idx = 0
    #TODO: Handle the error case if the RFI length is 0
    if(RFI_Start_Found(time_array, single_freq_power_array, idx, time_slope_threshold)):     #If I found the starting pattern
        RFI_start_value = single_freq_power_array[idx]
        rfi_start_idx = idx
        rfi_minimum_length = 10     #This exists to avoid the algorithm from fake crossing near the start due to fluctuations.
        for end_idx in range(idx + rfi_minimum_length, last_idx):                                   #Finish looking at the bandwidth interval looking for the end
            # if(Spectrum_RFI_End_Found(time_array, spectrum_power_array, rfi_end_idx, slope_threshold)):     #If I found the ending pattern          TODO: I have a problem in the end found algorithm
            if(RFI_End_Found(single_freq_power_array, end_idx, RFI_start_value)):     #If the RFI crosses below the starting value         TODO: I have a problem in the end found algorithm
                possible_RFI_Found = True   
                rfi_end_idx = end_idx
                break
                # RFI_spectral_thickness = (rfi_end_idx - rfi_start_idx)
        if(possible_RFI_Found == False):
            print("RFI ending not found")
    return possible_RFI_Found, rfi_start_idx, rfi_end_idx

def RFI_Time_Scan(time_array, single_freq_power_array, freq_idx, time_slope_threshold): #NOTE: I might have issues if I have a nan value on my array
    possible_RFI_events = []
    for rfi_start_idx in range(0, (len(time_array) - 1)):
        possible_RFI_Found, possible_RFI_Time, possible_RFI_duration = Possible_RFI_Found(time_array, single_freq_power_array, rfi_start_idx, time_slope_threshold)
        if(possible_RFI_Found):
            possible_RFI_events.append([possible_RFI_Time, possible_RFI_duration])
    return possible_RFI_events  #Returns array of [RFI_time_idx, RFI_duration_idx] TODO: in idxes or actual time?

def RFI_Start_Found(time_array, spectrum_power_array, idx, slope_threshold):
    RFI_Start_Found = False
    spectrum_gradient = np.gradient(spectrum_power_array, 3)
    current_slope = spectrum_gradient[idx-1]
    next_slope = spectrum_gradient[idx]
    if((current_slope <= slope_threshold) and (next_slope >= slope_threshold)):
        RFI_Start_Found = True
        # print("RFI start information:\n - current slope:", current_slope,"next slope:", next_slope, "Slope threshold:", slope_threshold)
    return RFI_Start_Found


def RFI_End_Found(spectrum_power_array, rfi_end_idx, RFI_start_value):
    retVal = False
    if spectrum_power_array[rfi_end_idx] <= RFI_start_value:
        retVal = True
    return retVal
        
def RFI_Spectrum_Scan(time_array, spectrum_power_array, freq_idx, slope_threshold): #NOTE: This might be an issue if I have a nan value on my array
    scan_bandwidth = 50 #[MHz]
    start_idx = int(freq_idx - int((scan_bandwidth/df)/2))  #This changes the bandwidth from MHz to idxes
    end_idx = int(freq_idx + int((scan_bandwidth/df)/2))
    RFI_confirmed = False
    RFI_spectral_thickness = 0
    for rfi_start_idx in range(start_idx, end_idx):                                             #Scan over the entire bandwith interval
        if(RFI_Start_Found(time_array, spectrum_power_array, rfi_start_idx, slope_threshold)):     #If I found the starting pattern
            RFI_start_value = spectrum_power_array[rfi_start_idx]
            rfi_minimum_length = 10     #This exists to avoid the algorithm from fake crossing near the start due to fluctuations.
            for rfi_end_idx in range(rfi_start_idx + rfi_minimum_length, end_idx):                                   #Finish looking at the bandwidth interval looking for the end
                # if(Spectrum_RFI_End_Found(time_array, spectrum_power_array, rfi_end_idx, slope_threshold)):     #If I found the ending pattern          TODO: I have a problem in the end found algorithm
                if(RFI_End_Found(spectrum_power_array, rfi_end_idx, RFI_start_value)):     #If the RFI crosses below the starting value         TODO: I have a problem in the end found algorithm
                    RFI_confirmed = True      
                    break
                    # RFI_spectral_thickness = (rfi_end_idx - rfi_start_idx)
                else:
                    continue
            if(RFI_confirmed):                                                                          #and stop looping throught the interval for efficiency
                break
        else:
            rfi_end_idx = end_idx
    return RFI_confirmed, rfi_start_idx, (rfi_end_idx)#RFI_spectral_thickness
    
def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx



###################################################################################################################################################################
# in_power_array        Input power array (RR, LL, reRL, imRL) at a single frequency TODO: This needs to change to 
# numt                  Number of time points to include in the 'window' in which the standard deviation is calculated
# threshold             Array of standard deviation thresholds to use in each stage of the RFI excision  

def Get_Polarized_Set(reRL_set, imRL_set, freq_measured):
    polarized = []
    reRL_set_in_use = reRL_set[:,freq_measured]    #TODO: maybe I can try making something general that I can scan over all frequencies
    imRL_set_in_use = imRL_set[:,freq_measured]
    for i,j in zip(reRL_set_in_use,imRL_set_in_use):
        PI = np.sqrt((i**2)+(j**2))
        polarized.append(PI)
    return np.array(polarized)


def Apply_RFI_Excision_Mask(mask, input_array): #TODO: This needs to be updated to implement all frequencies
    
    masked_copy = input_array.copy()
    for idx in range(0, len(input_array)-1):
        if(mask[idx] == 1):
            masked_copy[idx] = np.nan
    return(masked_copy)

def Combine_Into_RFI_Events(possible_RFI):
    Combined_RFI_events = [] 
    remaining_idxes = len(possible_RFI)

    current_idx = 1
    rfi_start = possible_RFI[current_idx]
    rfi_duration = 1

    while remaining_idxes >= 2:
        remaining_idxes = (len(possible_RFI) - current_idx)
        previous_idx = current_idx - 1
        idx_gap = (possible_RFI[current_idx] - possible_RFI[previous_idx])
        if(idx_gap <= 2):
            rfi_duration += 1
            current_idx += 1
        else:
            #Close current event
            Combined_RFI_events.append([rfi_start, rfi_duration])
            #Initiate next event starting on the current_idx
            rfi_duration = 1
            current_idx += 1
            rfi_start = possible_RFI[current_idx]

    return Combined_RFI_events #[idx, idx_duration]

In [6]:
def Find_RFI_Regions(time_array, power_array, time_steps, time_slope_threshold, freq_slope_threshold, starting_frequency, abs_threshold):
    confirmed_RFI_events = []
    # mask = np.zeros([len(time_array)])  #Initiate empty mask
    # freq_measured = np.min(np.where(abs(freq-starting_frequency)<df))#[0][0] #[MHz]
    freq_idx = find_nearest_idx(freq, starting_frequency)
    freq_measured = freq[freq_idx]
    print("input starting frequency:", starting_frequency)
    print("closest frequency measured:", freq_measured)


    possible_RFI_idxes = np.where(power_array[:, freq_idx] >= abs_threshold)
    combined_possible_RFI = Combine_Into_RFI_Events(possible_RFI_idxes[0])  #NOTE:
    print("combined_possible_RFI before Time_Scan", combined_possible_RFI)


    possible_RFI_events = RFI_Time_Scan(time_array, power_array[:, freq_idx], time_steps, time_slope_threshold) #Returns an array with [idx, RFI_idx_duration] of candidate RFI signals
    print("possible RFI events:", len(possible_RFI_events))
    # for event in range(len(possible_RFI_events)):
    #     dt = (time_array[1] - time_array[0])
    #     RFI_event_time_idx = possible_RFI_events[event][0]              #in [idxes]
    #     RFI_event_duration = possible_RFI_events[event][1]*(dt)         #in [time_units]
    #     #Confirms by doing a small spectrum scan
    #     RFI_event_confirmation_result = RFI_Spectrum_Scan(time_array, power_array[RFI_event_time_idx, :], freq_idx, slope_threshold) #Returns [bools, rfi_start_idx, rfi_end_idx]_______________old:[bool, rfi_start_idx, idx_spectral_thickness]
    #     # print("RFI", event ,"starts at:", freq[RFI_event_confirmation_result[1]],"ends at:", freq[RFI_event_confirmation_result[2]])
    #     if(RFI_event_confirmation_result[0]):
    #         print("RFI", event, "was confirmed")
    #         tol = 100
    #         mid_time_idx = possible_RFI_events[event][0]
    #         RFI_duration_result = Find_RFI_Duration(time_array, power_array, mid_time_idx, tol, freq_idx, slope_threshold)
    #         print("RFI[", event,"] duration results", RFI_duration_result)
    #         # confirmed_RFI_events.append([freq_measured, RFI_event_confirmation_result[1]*df, time_array[possible_RFI_events[event][0]], RFI_event_duration])
    #         # confirmed_RFI_events.append([freq[RFI_event_confirmation_result[1]], (RFI_event_confirmation_result[2] - RFI_event_confirmation_result[1])*df, time_array[possible_RFI_events[event][0]], RFI_event_duration])
    #         confirmed_RFI_events.append([freq[RFI_event_confirmation_result[1]], (RFI_event_confirmation_result[2] - RFI_event_confirmation_result[1])*df, time_array[RFI_duration_result[0]], RFI_duration_result[1]*dt])
    #         #TODO: Run the RFI_Spectrum_Scan_Sweep to find the borders of the RFI duration
    # print("confirmed RFI events:", len(confirmed_RFI_events))

    #Apply algorithm to determine duration of RFI
    return combined_possible_RFI #An Array with [freq, df, time, dt] for each confirmed RFI
    # return confirmed_RFI_events #An Array with [freq, df, time, dt] for each confirmed RFI

In [7]:
rfi_region_chosen = 0

time_steps = 30
time_slope_threshold = 1e5 #This number needs to be adjusted in order to correctly filter out non-RFI
freq_slope_threshold = 1e5 #This number needs to be adjusted in order to correctly filter out non-RFI
freq_chosen = 842
abs_threshold = 2e6
polarized_Set_RFI_Regions = Find_RFI_Regions(t_plt, polarized_set, time_steps, time_slope_threshold, freq_slope_threshold, freq_chosen, abs_threshold)
RFI_region = polarized_Set_RFI_Regions[rfi_region_chosen]


time_framing = 2#25

rfi_start_idx = RFI_region[0]
rfi_end_idx = RFI_region[0] + RFI_region[1]
rfi_start_idx_framed = RFI_region[0] #- RFI_region[1]
rfi_end_idx_framed =  RFI_region[0] + RFI_region[1]*time_framing

print("rfi_start_idx_framed", rfi_start_idx_framed)
print("rfi_end_idx_framed",rfi_end_idx_framed)
# PLOTTING THE CROSS-SECTIONS ------------------------------------------------------------------------------------------------
freq_idx = find_nearest_idx(freq, freq_chosen)
rfi_halfway_time = int((rfi_start_idx + rfi_end_idx)/2)
fig,axs1 = plt.subplots(1,1,figsize=(15,10)) 

# axs1[0].plot(t_plt[rfi_start_idx_framed: rfi_end_idx_framed], LL_set[rfi_start_idx_framed: rfi_end_idx_framed, freq_idx], label = 'LL_set')
# axs1[0].plot(t_plt[rfi_start_idx: rfi_end_idx], LL_set[rfi_start_idx: rfi_end_idx, freq_idx], label = 'RFI Selected')




# axs1[0][0].plot(t_plt[rfi_start_idx_framed: rfi_end_idx_framed], polarized_set[rfi_start_idx_framed: rfi_end_idx_framed, freq_idx], label = 'polarized_set')
noise_idx = np.where(noise_set == 1)
polarized_set[noise_idx, :] = np.nan
# pol_set_masked = Apply_Mask(polarized_set, noise_set)

# axs1.plot(t_plt[rfi_start_idx: rfi_end_idx], polarized_set[rfi_start_idx: rfi_end_idx, freq_idx], label = 'LL_set in dB')
axs1.plot(t_plt, polarized_set[:, freq_idx], label = 'LL_set in dB')
axs1.axhline(abs_threshold,xmin=0,xmax=3,c="red",linewidth=1,zorder=0)
axs1.axvline(t_plt[rfi_start_idx],ymin=0,ymax=3,c="black",linewidth=.5,zorder=0)
axs1.axvline(t_plt[rfi_end_idx],ymin=0,ymax=3,c="black",linewidth=.5,zorder=0)
axs1.set_ylim(ymax=1e7, ymin=0)


# axs1[0][0].plot(t_plt[rfi_start_idx: rfi_end_idx], polarized_set[rfi_start_idx: rfi_end_idx, freq_idx], label = 'RFI Selected')
# axs1[0][0].plot(t_plt[rfi_start_idx: rfi_end_idx], 10.*np.log10(LL_set[rfi_start_idx: rfi_end_idx, freq_idx].T), label = 'RFI Selected')
axs1.set_xlabel('Time (UTC)',fontsize=12)
axs1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs1.set_ylabel('Power',fontsize=12)
axs1.set_title("Polarized Time Cross Section")


input starting frequency: 842
closest frequency measured: 842.0208333333334


IndexError: index 264 is out of bounds for axis 0 with size 264

In [8]:
df = freq[1]-freq[0]

def DVA_Waterfall_View():
    #TODO: Add another DVA_Waterfall interactive_function such that I can change between LL_Set, RR_set, etc...
    power_min = 70
    power_max = 78

    fig,axs = plt.subplots(1,1,figsize=(15,10)) 
    fs = 16

    im = axs.imshow(10.*np.log10(LL_set.T),aspect='auto',vmin=power_min,vmax=power_max,origin='lower',
                extent=[t_plt[0],t_plt[-1],freq[0],freq[-1]])

    divider = make_axes_locatable(axs)
    cax = divider.append_axes('right', size='2%', pad=0.05)
    cbar = fig.colorbar(im, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=fs) 
    cbar.set_label('Power (dB)', fontsize=fs)

    axs.set_xlim(t_plt[0],t_plt[-1])
    axs.set_ylim(freq[0],freq[-1])
    axs.tick_params(axis='both', labelsize=fs)
    axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs.set_xlabel('Time (UTC)',fontsize=fs)        
    axs.set_ylabel('Frequency (MHz)',fontsize=fs)


def DVA_Cross_Sections(freq_chosen, time_idx, freq_cross_section):
    freq_measured = np.where(abs(freq-freq_chosen)<df)[0][0]

    fs = 12    
    fig,axs1 = plt.subplots(1,1,figsize=(16,6))  
    # TODO: use the log of LL_set_clean
    if(freq_cross_section):
        axs1.plot(freq, LL_set[time_idx,:], label = 'LL_set with polarized mask')
        axs1.vlines(freq_chosen, 0 , 100e9, color = 'red')
        axs1.set_ylim(np.min(LL_set[:,freq_measured]), np.max(LL_set[time_idx,:]))
        axs1.set_xlabel('Frequency',fontsize=fs)
        axs1.set_ylabel('Power',fontsize=fs)
    else:
        axs1.plot(t_plt, LL_set[:,freq_measured], label = 'LL_set unmasked RFI')
        axs1.vlines(t_plt[time_idx], 0 , 100e9, color = 'red')

        axs1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        axs1.set_ylim(np.min(LL_set[:,freq_measured]), np.max(LL_set[:,freq_measured]))
        axs1.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
        axs1.set_xlabel('Time (UTC)',fontsize=fs)
        axs1.set_ylabel('Power',fontsize=fs)
        axs1.legend()

def DVA_Visualization(waterfall_enabled):
    if waterfall_enabled:
        interact(DVA_Waterfall_View)
    else:
        interact(DVA_Cross_Sections, freq_chosen = (350, 1000, df), freq_cross_section = False, time_idx = (0,len(t_plt)-1))


interact(DVA_Visualization, waterfall_enabled = True)

#TODO: plot power in DB

interactive(children=(Checkbox(value=True, description='waterfall_enabled'), Output()), _dom_classes=('widget-…

<function __main__.DVA_Visualization(waterfall_enabled)>

# This is the flagging visualization

In [9]:
def Flagging_Visualization(rfi_region_chosen):
    time_steps = 30
    time_slope_threshold = 1e5 #This number needs to be adjusted in order to correctly filter out non-RFI
    freq_slope_threshold = 1e5 #This number needs to be adjusted in order to correctly filter out non-RFI
    freq_chosen = 845
    polarized_Set_RFI_Regions = Find_RFI_Regions(t_plt, polarized_set, time_steps, time_slope_threshold, freq_slope_threshold, freq_chosen) #Returns an array of [freq, df, time, dt]1
    
    RFI_region = polarized_Set_RFI_Regions[rfi_region_chosen]

    # print("real df is:", freq[1] - freq[0])

    #Try plotting the first RFI_Region

    power_min = 69
    power_max = 75

    time_framing = 2#25
    freq_framing = 2

    t1_plt = RFI_region[2]
    t2_plt = RFI_region[2] + RFI_region[3]
    t1_plt_framed = RFI_region[2] - RFI_region[3]
    t2_plt_framed = RFI_region[2] + RFI_region[3]*time_framing

    freq1 = RFI_region[0]
    freq2 = RFI_region[0] + RFI_region[1]
    freq1_framed = RFI_region[0] - RFI_region[1]
    freq2_framed = RFI_region[0] + RFI_region[1]*freq_framing
    ######################################
    rfi_start_idx = find_nearest_idx(t_plt, t1_plt)
    rfi_end_idx = find_nearest_idx(t_plt, t2_plt)
    rfi_start_idx_framed = find_nearest_idx(t_plt, t1_plt_framed)
    rfi_end_idx_framed = find_nearest_idx(t_plt, t2_plt_framed)
    print("rfi_start_idx_framed", rfi_start_idx_framed)
    print("rfi_end_idx_framed",rfi_end_idx_framed)

    f_plt_start_idx = find_nearest_idx(freq, freq1)
    f_plt_end_idx = find_nearest_idx(freq, freq2)
    f_plt_start_idx_framed = find_nearest_idx(freq, freq1_framed)
    f_plt_end_idx_framed = find_nearest_idx(freq, freq2_framed)


    # PLOTTING THE CROSS-SECTIONS ------------------------------------------------------------------------------------------------
    freq_idx = find_nearest_idx(freq, freq_chosen)
    rfi_halfway_time = int((rfi_start_idx + rfi_end_idx)/2)
    fig,axs1 = plt.subplots(2,2,figsize=(15,10)) 

    # axs1[0].plot(t_plt[rfi_start_idx_framed: rfi_end_idx_framed], LL_set[rfi_start_idx_framed: rfi_end_idx_framed, freq_idx], label = 'LL_set')
    # axs1[0].plot(t_plt[rfi_start_idx: rfi_end_idx], LL_set[rfi_start_idx: rfi_end_idx, freq_idx], label = 'RFI Selected')




    # axs1[0][0].plot(t_plt[rfi_start_idx_framed: rfi_end_idx_framed], polarized_set[rfi_start_idx_framed: rfi_end_idx_framed, freq_idx], label = 'polarized_set')
    10.*np.log10(LL_set[rfi_start_idx_framed: rfi_end_idx_framed, freq_idx].T)

    axs1[0][0].plot(t_plt[rfi_start_idx_framed: rfi_end_idx_framed], 10.*np.log10(LL_set[rfi_start_idx_framed: rfi_end_idx_framed, freq_idx].T), label = 'LL_set in dB')
    # axs1[0][0].plot(t_plt[rfi_start_idx: rfi_end_idx], polarized_set[rfi_start_idx: rfi_end_idx, freq_idx], label = 'RFI Selected')
    # axs1[0][0].plot(t_plt[rfi_start_idx: rfi_end_idx], 10.*np.log10(LL_set[rfi_start_idx: rfi_end_idx, freq_idx].T), label = 'RFI Selected')
    axs1[0][0].set_xlabel('Time (UTC)',fontsize=12)
    axs1[0][0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs1[0][0].set_ylabel('Power',fontsize=12)
    axs1[0][0].set_title("Polarized Gaussian Approach")

    axs1[0][1].plot(freq[f_plt_start_idx_framed: f_plt_end_idx_framed], polarized_set[rfi_halfway_time,f_plt_start_idx_framed: f_plt_end_idx_framed], '-ok', label = 'polarized_set', color = 'blue')
    axs1[0][1].plot(freq[f_plt_start_idx: f_plt_end_idx], polarized_set[rfi_halfway_time,f_plt_start_idx: f_plt_end_idx], '-ok', label = 'RFI Selected', color = 'orange')
    axs1[0][1].set_xlabel('Frequency',fontsize=12)
    axs1[0][1].set_ylabel('Power',fontsize=12)
    axs1[0][1].set_title("Spectrum at the center of the RFI duration")

    axs1[1][0].plot(freq[f_plt_start_idx_framed: f_plt_end_idx_framed], polarized_set[rfi_start_idx,f_plt_start_idx_framed: f_plt_end_idx_framed], '-ok', label = 'polarized_set', color = 'blue')
    axs1[1][0].plot(freq[f_plt_start_idx: f_plt_end_idx], polarized_set[rfi_start_idx,f_plt_start_idx: f_plt_end_idx], '-ok', label = 'RFI Selected', color = 'orange')
    axs1[1][0].set_xlabel('Frequency',fontsize=12)
    axs1[1][0].set_ylabel('Power',fontsize=12)
    axs1[1][0].set_title("Spectrum at the start of the RFI duration")

    axs1[1][1].plot(freq[f_plt_start_idx_framed: f_plt_end_idx_framed], polarized_set[rfi_end_idx,f_plt_start_idx_framed: f_plt_end_idx_framed], '-ok', label = 'polarized_set', color = 'blue')
    axs1[1][1].plot(freq[f_plt_start_idx: f_plt_end_idx], polarized_set[rfi_end_idx,f_plt_start_idx: f_plt_end_idx], '-ok', label = 'RFI Selected', color = 'orange')
    axs1[1][1].set_xlabel('Frequency',fontsize=12)
    axs1[1][1].set_ylabel('Power',fontsize=12)
    axs1[1][1].set_title("Spectrum at the end of the RFI duration")

    axs1[0][0].legend()
    axs1[1][0].legend()
    axs1[0][1].legend()
    axs1[1][1].legend()
    #--------------------------------------------------------------------------------------------------------------------------


    # PLOTTING WATERFALL RFI ------------------------------------------------------------------------------------------------------
    RFI_duration_time = t_plt[rfi_start_idx_framed: rfi_end_idx_framed]
    RFI_freq_bandwith = freq[f_plt_start_idx_framed: f_plt_end_idx_framed]


    fig,axs = plt.subplots(1,1,figsize=(15,10)) 
    fs = 16

    # Create a Rectangle patch ---------------------------------------------------------------------------------------------------
    RFI_detected = patches.Rectangle((t_plt[rfi_start_idx], freq[f_plt_start_idx]), np.abs(t_plt[rfi_end_idx] - t_plt[rfi_start_idx]), np.abs(freq[f_plt_end_idx] - freq[f_plt_start_idx]), linewidth=1.5, edgecolor='r', facecolor='none')
    # Add the patch to the Axes0
    axs.add_patch(RFI_detected)
    # ---------------------------------------------------------------------------------------------------------------------------

    im = axs.imshow(10.*np.log10(polarized_set[rfi_start_idx_framed: rfi_end_idx_framed,f_plt_start_idx_framed: f_plt_end_idx_framed].T),aspect='auto',vmin=power_min,vmax=power_max,origin='lower',
                extent=[RFI_duration_time[0],RFI_duration_time[-1],RFI_freq_bandwith[0],RFI_freq_bandwith[-1]])

    divider = make_axes_locatable(axs)
    cax = divider.append_axes('right', size='2%', pad=0.05)
    cbar = fig.colorbar(im, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=fs) 
    cbar.set_label('Power (dB)', fontsize=fs)

    axs.set_xlim(RFI_duration_time[0],RFI_duration_time[-1])
    # axs.set_ylim(RFI_freq_bandwith[0],RFI_freq_bandwith[-1])
    axs.tick_params(axis='both', labelsize=fs)
    axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs.set_xlabel('Time (UTC)',fontsize=fs)        
    axs.set_ylabel('Frequency (MHz)',fontsize=fs)


    #TODO: I don't think it's clear which region this method is flaggin as RFI


# Flagging_Visualization(0)

interact(Flagging_Visualization, rfi_region_chosen = (0,10))

interactive(children=(IntSlider(value=5, description='rfi_region_chosen', max=10), Output()), _dom_classes=('w…

<function __main__.Flagging_Visualization(rfi_region_chosen)>

# Future notes:

I think the algorighm should use primarily the frequency sweep to determine the shape of the RFI.

steps:
- Work on the frequency bandwidth algorithm
- Repeat that frequency sweep over time and use it to determine the duration of the RFI, since the "gaussian" shape of the RFI is very unreliable


### Anna's comment:
- Ignore the noise beacon using the file. There's an notebook that does that already